In [1]:
import copy
import sys
import os
from functools import partial

import geopandas as gpd
from pandas import IndexSlice as idx
import numpy as np
import tensorflow as tf
import gpflow
from tensorflow.python.data.ops.iterator_ops import OwnedIterator as DatasetOwnedIterator

#from metrics import fixed_top_X
#from model_runner import run_adam

code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)

from perturbations import perturbed
from bpr import bpr_variable_k_no_ties

from onoffgpf import OnOffSVGP, OnOffSVGPPoiMC, OnOffLikelihood,OnOffSVGPBatch
gpflow.config.default_float()

In [3]:
data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/'

In [4]:
data_path = os.path.join(data_dir, 'clean_annual_tract')
timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'
last_train_year = 2018
first_train_year = 2000
test_years = 2
use_auto = False
use_svi = True
seed=360
inducing_points = 200
learning_rate = 0.001
minibatch_size = 100

sigma = 0.05
bpr_samples = 37
noise='normal'

In [162]:
x_idx_cols = [geography_col, 'lat','lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile','year',
              'neighbor_t', 'self_t-1']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['lat','lon', timestep_col,
                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                 'svi_pctile',
                 'neighbor_t', 'self_t-1']

data_gdf = gpd.read_file(data_path)

train_x = data_gdf[(data_gdf['year'] <= last_train_year) &
                   (data_gdf['year'] >= first_train_year)][x_idx_cols]
train_y = data_gdf[(data_gdf['year'] <= last_train_year) &
                   (data_gdf['year'] >= first_train_year)][y_idx_cols]
test_x = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][x_idx_cols]
test_y = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][y_idx_cols]

num_data = train_x.shape[0]
num_latent = 1

In [163]:
time_window = 10
first_train_eval_year = 2014
last_train_eval_year = 2018

In [164]:
multiindexed_gdf = data_gdf.set_index(['geoid','year'])
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(features_only))

timestep_feature_idx = features_only.index('timestep')

xs =[]
ys = []

for eval_year in range(first_train_eval_year, last_train_eval_year+1):
    
    train_x_df = multiindexed_gdf.loc[idx[:,eval_year-time_window:eval_year-1], features_only]
    train_y_df = multiindexed_gdf.loc[idx[:,eval_year], 'deaths']
    
    train_x_vals = train_x_df.values.reshape(train_shape)
    # make sure we did reshape right way
    assert((train_x_vals[:,4,timestep_feature_idx]==train_x_vals[0,4,timestep_feature_idx]).all())
    train_y_vals = train_y_df.values
    
    xs.append(train_x_vals)
    ys.append(train_y_vals)
    
    
x_BSTD = np.stack(xs,axis=0)
y_BS = np.stack(ys)

x_BSTD = tf.convert_to_tensor(x_BSTD, dtype=tf.float32)
y_BS = tf.convert_to_tensor(y_BS, dtype=tf.float32)

B, S, T, D = x_BSTD.shape

assert(B==len( range(first_train_eval_year, last_train_eval_year+1)))
assert(S==num_geoids)
assert(T==time_window)
assert(D==len(features_only))

# Reshape the training data to flatten the dimensions
x_BSF_flat = tf.reshape(x_BSTD, (B, S, T * D), )

In [176]:
def top_k_idx(input_BD, **kwargs):
    
    _, idx_BD = tf.math.top_k(input_BD, **kwargs)
    input_depth = input_BD.shape[-1]
    one_hot_idx_BKD = tf.one_hot(idx_BD, input_depth)
    #Sum over k dimension so we dont have to worry about sorting
    k_hot_idx_BD = tf.reduce_sum(one_hot_idx_BKD, axis=-2)
    
    
    return k_hot_idx_BD
top_100_idx = partial(top_k_idx, k=100)
perturbed_top_100 = perturbed(top_100_idx,
                         num_samples=bpr_samples,
                         sigma=sigma,
                         noise=noise,
                         batched=True)

class PerturbedBPRModel(tf.keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            top_100_indicators = perturbed_top_100(y_pred)
            true_top_100_val, true_top_100_idx = tf.math.top_k(y,k=100)
            
            denominator = tf.reduce_sum(true_top_100_val, axis=-1)
            numerator = tf.reduce_sum(top_100_indicators*y, axis=-1)
            
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(numerator, denominator, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [177]:
# Create a TensorFlow model
linear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(T, input_shape=(S,T * D,), activation='relu',
                          ),
    tf.keras.layers.Dense(1, input_shape=(S,T ), activation=None,
                          )
])

In [178]:
inputs = tf.keras.Input(shape=(S,T*D))
outputs = tf.squeeze(linear_model(inputs),axis=-1)

In [179]:
model = PerturbedBPRModel(inputs, outputs)

In [186]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

In [187]:
# Compile the model
def weird_loss(a,b):
    return -a/b

model.compile(optimizer=optimizer, loss=weird_loss)



In [188]:
# Train the model
model.fit(x_BSF_flat, y_BS, epochs=1000, batch_size=5)  

Epoch 1/1000
1/1 [==============================] - 0s 468ms/step - loss: -0.2243
Epoch 2/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.2085
Epoch 3/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.2019
Epoch 4/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.1980
Epoch 5/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.1990
Epoch 6/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.2011
Epoch 7/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.1999
Epoch 8/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.1990
Epoch 9/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.2037
Epoch 10/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.2072
Epoch 11/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.2126
Epoch 12/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.2146
Epoch 13/1000
1/1 [=====

Epoch 101/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3043
Epoch 102/1000
1/1 [==============================] - 0s 48ms/step - loss: -0.3042
Epoch 103/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3012
Epoch 104/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3045
Epoch 105/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.3016
Epoch 106/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3035
Epoch 107/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.3040
Epoch 108/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3029
Epoch 109/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3040
Epoch 110/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3058
Epoch 111/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3021
Epoch 112/1000
1/1 [==============================] - 0s 48ms/step - loss: -0.3045
Epoc

1/1 [==============================] - 0s 46ms/step - loss: -0.3406
Epoch 298/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3392
Epoch 299/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3403
Epoch 300/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3400
Epoch 301/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3400
Epoch 302/1000
1/1 [==============================] - 0s 48ms/step - loss: -0.3401
Epoch 303/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3410
Epoch 304/1000
1/1 [==============================] - 0s 48ms/step - loss: -0.3414
Epoch 305/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3415
Epoch 306/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.3403
Epoch 307/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.3410
Epoch 308/1000
1/1 [==============================] - 0s 48ms/step - loss: -0.3410
Epoch 309/1000
1/1 

1/1 [==============================] - 0s 46ms/step - loss: -0.3436
Epoch 494/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3445
Epoch 495/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3434
Epoch 496/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3448
Epoch 497/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.3442
Epoch 498/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3446
Epoch 499/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3471
Epoch 500/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3467
Epoch 501/1000
1/1 [==============================] - 0s 48ms/step - loss: -0.3456
Epoch 502/1000
1/1 [==============================] - 0s 48ms/step - loss: -0.3468
Epoch 503/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3455
Epoch 504/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.3465
Epoch 505/1000
1/1 

1/1 [==============================] - 0s 47ms/step - loss: -0.3480
Epoch 690/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3474
Epoch 691/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3478
Epoch 692/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3475
Epoch 693/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3483
Epoch 694/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3473
Epoch 695/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3450
Epoch 696/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3465
Epoch 697/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3463
Epoch 698/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3474
Epoch 699/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3446
Epoch 700/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3449
Epoch 701/1000
1/1 

1/1 [==============================] - 0s 48ms/step - loss: -0.3133
Epoch 886/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3150
Epoch 887/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3146
Epoch 888/1000
1/1 [==============================] - 0s 48ms/step - loss: -0.3167
Epoch 889/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3152
Epoch 890/1000
1/1 [==============================] - 0s 50ms/step - loss: -0.3167
Epoch 891/1000
1/1 [==============================] - 0s 52ms/step - loss: -0.3149
Epoch 892/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3162
Epoch 893/1000
1/1 [==============================] - 0s 45ms/step - loss: -0.3178
Epoch 894/1000
1/1 [==============================] - 0s 44ms/step - loss: -0.3169
Epoch 895/1000
1/1 [==============================] - 0s 46ms/step - loss: -0.3179
Epoch 896/1000
1/1 [==============================] - 0s 47ms/step - loss: -0.3174
Epoch 897/1000
1/1 

In [110]:
y_pred = model(x_BSF_flat, training=True)  # Forward pass
top_100_indicators = perturbed_top_100(y_pred)
true_top_100_val, true_top_100_idx = tf.math.top_k(y_BS,k=100)

denominator = tf.reduce_sum(true_top_100_val, axis=-1)
numerator = tf.reduce_sum(top_100_indicators*y_pred, axis=-1)

# Compute the loss value
# (the loss function is configured in `compile()`)
loss = model.compiled_loss(numerator, denominator, regularization_losses=model.losses)

In [112]:
numerator

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([5243.3545, 5301.8384, 5358.162 , 5417.177 , 5473.1704],
      dtype=float32)>

In [178]:
top_100 = partial(tf.math.top_k, k=100)
perturbed_top_100 = perturbed(top_100,
                         num_samples=bpr_samples,
                         sigma=sigma,
                         noise='normal',
                         batched=True)

In [188]:
sample_preds = tf.squeeze(model(tf.expand_dims(x_BSF_flat[0,:,:],axis=0)), axis=-1)

In [191]:
perturbed_top_100(sample_preds)

InvalidArgumentError: cannot compute Pack as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:Pack] name: packed

In [190]:
sample_preds

<tf.Tensor: shape=(1, 1620), dtype=float32, numpy=
array([[ 0.08766611, -0.10226385,  0.05699642, ...,  0.5287748 ,
         0.09916075,  0.12338884]], dtype=float32)>

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x.loc[:, features_only].values,
                                                    train_y.loc[:, outcome_col].values.reshape(-1, 1))).repeat().shuffle(num_data)
train_iter = iter(train_dataset.batch(minibatch_size))

2023-05-18 06:10:35.626976: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [30780,10]
	 [[{{node Placeholder/_0}}]]
2023-05-18 06:10:35.627359: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [30780,10]
	 [[{{node Placeholder/_0}}]]


In [ ]:
def perturbed_loss_on_batch(model, data_batch):
    KL = model.build_prior_KL()
    
    X, Y = data_batch
    
    gfmean, gfvar, gfmeanu, _, _, _, _, _, _ = self.build_predict(X)

    y_pred = gfmean
    

In [ ]:
def custom_perturbed_loss(model, compile=True):
    
    
    training_loss = partial 
    
    if isinstance(data, DatasetOwnedIterator):
        if compile:
            # lambda because: https://github.com/GPflow/GPflow/issues/1929
            training_loss_lambda = lambda d: self.training_loss(d)
            input_signature = [data.element_spec]
            training_loss = tf.function(training_loss_lambda, input_signature=input_signature)

        def closure() -> tf.Tensor:
            assert isinstance(data, DatasetOwnedIterator)  # Hint for mypy.
            batch = next(data)
            return training_loss(batch)
    

In [42]:
training_loss = model.training_loss_closure(train_iter, compile=True)


In [44]:
@tf.function
def optimization_step():
    optimizer.minimize(training_loss, model.trainable_variables)


In [61]:
logf = []
for step in range(10000):
    optimization_step()
    if step % 10 == 0:
        elbo = -training_loss().numpy()
        logf.append(elbo)

In [57]:
training_loss()

<tf.Tensor: shape=(), dtype=float64, numpy=908538.6419173388>

In [85]:
perturbed_bpr(tf.expand_dims(tf.constant([.3, .2, .1, .5, .4]), axis=0), 
              tf.expand_dims(tf.constant([.3, .2, .1, .5, .4]), axis=0))

InvalidArgumentError: {{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} params.shape[0]: 10 should be equal to indices.shape[0]: 1 [Op:GatherV2]

In [87]:
np.expand_dims(np.array(range(1000),dtype=np.float32),-1).shape

(1000, 1)

In [78]:
def bpr_variable_k_no_ties(y_true, y_pred, k=None):
    """Calculate BPR-k often not used due to loss functions expected to only take 2 arguments

    Args:
        y_true: True outcome
        y_pred: predicted outcome
        k (int): Threshold for BPR. Required, but defaults to None because partial re

    Returns:
        The BPR-k score

    Note: This method DOES NOT handle ties, as it is meant to be used in a perturbed fashion
    """

    _, top_k_pred_idx = tf.math.top_k(y_pred, k=k)
    top_k_true_val, top_k_true_idx = tf.math.top_k(y_true, k=k)

    # Denominator is actual top-k
    # Impossible to have ties here, a tie wouldn't change the value
    denominator = tf.reduce_sum(top_k_true_val, axis=-1)

    # Numerator is sum of true values at the locations indicated by predictions
    # Note: there could be ties here. We choose to ignore and deal with noise
    batch_dims = min(1, tf.rank(y_true)-1)
    true_val_at_pred_top_k = tf.gather(y_true, top_k_pred_idx, batch_dims=batch_dims, axis=-1)
    numerator = tf.reduce_sum(true_val_at_pred_top_k, axis=-1)

    bpr_k_value = numerator / denominator

    return bpr_k_value



In [146]:
input_tensor = tf.expand_dims(tf.constant([[.3, .2, .1, .5, .4],[.3, .2, .1, .5, .4]]), axis=0)

In [147]:
  input_shape = tf.shape(input_tensor)  # [B, D1, ... Dk], k >= 1
  perturbed_input_shape = tf.concat([[bpr_samples], input_shape], axis=0)

  noises = sample_noise_with_gradients('normal', perturbed_input_shape)
  additive_noise, noise_gradient = tuple(
      [tf.cast(noise, dtype=input_tensor.dtype) for noise in noises])
  perturbed_input = tf.expand_dims(input_tensor, 0) + sigma * additive_noise

  # [N, B, D1, ..., Dk] -> [NB, D1, ..., Dk].
  flat_batch_dim_shape = tf.concat([[-1], input_shape[1:]], axis=0)
  perturbed_input = tf.reshape(perturbed_input, flat_batch_dim_shape)
  # Calls user-defined function in a perturbation agnostic manner.
  perturbed_output = func(perturbed_input, *args, **kwargs)
  # [NB, D1, ..., Dk] ->  [N, B, D1, ..., Dk].
  perturbed_input = tf.reshape(perturbed_input, perturbed_input_shape)
  # Either
  #   (Default case): [NB, D1, ..., Dk] ->  [N, B, D1, ..., Dk]
  # or
  #   (Full-reduce case) [NB] -> [N, B]
  perturbed_output_shape = tf.concat(
      [[num_samples], [-1], tf.shape(perturbed_output)[1:]], axis=0)
  perturbed_output = tf.reshape(perturbed_output, perturbed_output_shape)

NameError: name 'func' is not defined

In [29]:
_GUMBEL = 'gumbel'
_NORMAL = 'normal'
SUPPORTED_NOISES = (_GUMBEL, _NORMAL)
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp

def sample_noise_with_gradients(
    noise, shape):
  """Samples a noise tensor according to a distribution with its gradient.

  Args:
   noise: (str) a type of supported noise distribution.
   shape: tf.Tensor<int>, the shape of the tensor to sample.

  Returns:
   A tuple Tensor<float>[shape], Tensor<float>[shape] that corresponds to the
   sampled noise and the gradient of log the underlying probability
   distribution function. For instance, for a gaussian noise (normal), the
   gradient is equal to the noise itself.

  Raises:
   ValueError in case the requested noise distribution is not supported.
   See perturbations.SUPPORTED_NOISES for the list of supported distributions.
  """
  if noise not in SUPPORTED_NOISES:
    raise ValueError('{} noise is not supported. Use one of [{}]'.format(
        noise, SUPPORTED_NOISES))

  if noise == _GUMBEL:
    sampler = tfp.distributions.Gumbel(0.0, 1.0)
    samples = sampler.sample(shape)
    gradients = 1 - tf.math.exp(-samples)
  elif noise == _NORMAL:
    sampler = tfp.distributions.Normal(0.0, 1.0)
    samples = sampler.sample(shape)
    gradients = samples

  return samples, gradients

In [139]:
perturbed_input_shape

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([10,  2,  5], dtype=int32)>

In [148]:
perturbed_bpr(perturbed_input,
             tf.constant([[.3, .2, .1, .5, .4],[.3, .2, .1, .5, .4]]))

InvalidArgumentError: {{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} params.shape[0]: 100 should be equal to indices.shape[0]: 2 [Op:GatherV2]

In [156]:
    #y_pred = tf.expand_dims(perturbed_input,axis=0)
    y_pred = perturbed_input
    y_true =  tf.expand_dims(tf.constant([[.3, .2, .1, .5, .4],[.3, .2, .1, .5, .4]]), axis=0)
    
    _, top_k_pred_idx = tf.math.top_k(y_pred, k=3)
    top_k_true_val, top_k_true_idx = tf.math.top_k(y_true, k=3)

    # Denominator is actual top-k
    # Impossible to have ties here, a tie wouldn't change the value
    denominator = tf.reduce_sum(top_k_true_val, axis=-1)

    # Numerator is sum of true values at the locations indicated by predictions
    # Note: there could be ties here. We choose to ignore and deal with noise
    batch_dims = min(1, tf.rank(y_true)-1)
    true_val_at_pred_top_k = tf.gather_nd(y_true, top_k_pred_idx, batch_dims=2,)
    numerator = tf.reduce_sum(true_val_at_pred_top_k, axis=-1)

    bpr_k_value = numerator / denominator

InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input to reshape is a tensor with 60 values, but the requested shape has 6 [Op:Reshape]

In [157]:
y_true.shape

TensorShape([1, 2, 5])

In [158]:
y_pred.shape

TensorShape([10, 2, 5])

In [109]:
tf.rank(y_true)

<tf.Tensor: shape=(), dtype=int32, numpy=3>

In [152]:
batch_dims 

1

In [47]:
bpr_k_value

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[1.       , 1.       , 1.       , 0.9166666, 1.       , 1.       ,
        1.       , 0.9166666, 1.       , 0.9166666]], dtype=float32)>

In [55]:
bpr_k_value

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[1.       , 1.       , 1.       , 0.9166666, 1.       , 1.       ,
        1.       , 0.9166666, 1.       , 0.9166666]], dtype=float32)>

In [18]:
gpd.IndexSlice

AttributeError: module 'geopandas' has no attribute 'IndexSlice'

geoid        year
25001010100  2004    0.0
             2005    2.0
             2006    0.0
             2007    0.0
             2008    0.0
                    ... 
25027761402  2010    0.0
             2011    0.0
             2012    0.0
             2013    0.0
             2014    0.0
Name: deaths, Length: 17820, dtype: float64

In [72]:
model.metrics